In [75]:
import pandas as pd
from heapq import nlargest
import matplotlib.pyplot as plt
import datetime as dt   
import calendar

# 1:

df_calls = pd.read_csv('calls.csv')
df_internet = pd.read_csv('internet.csv')
df_messages = pd.read_csv('messages.csv')
df_tariffs = pd.read_csv('tariffs.csv')
df_users = pd.read_csv('users.csv')
df_internet.drop(columns = ['Unnamed: 0'] ,axis = 1, inplace=True)
#df_users['churn_date'].value_counts()

In [76]:
# 2.1.1 Кол-во сделанных звонков и израсходованных минут по месяцам:
df_callsUser = df_calls['user_id'].value_counts().to_frame()
df_callsUser
df_callsUser = df_callsUser.reset_index()
df_callsUser.rename(columns={"index": 'num_of_call'}, inplace=True)
df_callsUser['1'] = df_callsUser['user_id']
df_callsUser['user_id'] = df_callsUser['num_of_call']
df_callsUser['num_of_call'] = df_callsUser['1']
df_callsUser.drop(columns = ['1'] ,axis = 1, inplace=True)


# 2.1.2 
df_calls_sum = pd.pivot_table(df_calls,
              index='user_id',
              values='duration',
              aggfunc='sum')['duration']
df_calls_sum = df_calls_sum.reset_index()
df_calls_sum.rename(columns={"duration": 'calls_sum'}, inplace=True)
# Наконец-то, нужно было ресетнуть и индекс
# По месяцам я эту работу делать не буду, чтобы не растягивать таблицу

df_users = df_users.merge(df_callsUser, how='left')
df_users = df_users.merge(df_calls_sum, how='left')
df_users
del df_callsUser
del df_calls_sum

In [85]:
# 2.2
df_messagesUser = df_messages['user_id'].value_counts().to_frame()
df_messagesUser = df_messagesUser.reset_index()
df_messagesUser.rename(columns={"user_id": 'num_of_mess'}, inplace=True)
df_messagesUser.rename(columns={"index": 'user_id'}, inplace=True)
df_users = df_users.merge(df_messagesUser, how='left')
del df_messagesUser

,id,mb_used,session_date,user_id
0,1000_0,112.95,2018-11-25,1000
1,1000_1,1052.81,2018-09-07,1000
2,1000_2,1197.26,2018-06-25,1000
3,1000_3,550.27,2018-08-22,1000
4,1000_4,302.56,2018-09-24,1000
...,...,...,...,...
149391,1499_152,318.90,2018-10-03,1499
149392,1499_153,490.13,2018-12-14,1499
149393,1499_154,0.00,2018-10-27,1499
149394,1499_155,1246.32,2018-11-26,1499


In [87]:
# 2.3:  Обьём, а не кол-во
df_internet_sum = pd.pivot_table(df_internet,
              index='user_id',
              values='mb_used',
              aggfunc='sum')['mb_used']
df_internet_sum = df_internet_sum.reset_index()
df_users = df_users.merge(df_internet_sum, how='left')
df_users

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff,num_of_call,calls_sum,num_of_mess,num_of_internet,mb_used
0,1000,52,NaN,Краснодар,Рафаил,Верещагин,2018-05-25,ultra,368.0,2418.24,496.0,211.0,107392.31
1,1001,41,NaN,Москва,Иван,Ежов,2018-11-01,smart,122.0,799.46,NaN,96.0,32466.00
2,1002,59,NaN,Стерлитамак,Евгений,Абрамович,2018-06-17,smart,217.0,1431.17,70.0,369.0,116036.52
3,1003,23,NaN,Москва,Белла,Белякова,2018-08-17,ultra,520.0,3600.52,380.0,104.0,57280.64
4,1004,68,NaN,Новокузнецк,Татьяна,Авдеенко,2018-05-14,ultra,176.0,921.21,1192.0,300.0,149039.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,1495,65,NaN,Иркутск,Авксентий,Фокин,2018-08-28,ultra,565.0,3688.57,340.0,254.0,127410.56
496,1496,36,NaN,Вологда,Трифон,Блохин,2018-01-27,smart,828.0,5661.37,468.0,834.0,270942.29
497,1497,32,NaN,Челябинск,Каролина,Агеева,2018-10-09,smart,201.0,1324.68,51.0,121.0,45574.97
498,1498,68,2018-10-25,Владикавказ,Всеволод,Акимчин,2018-07-19,smart,180.0,1221.60,168.0,232.0,74425.22
